In [1]:
import math
import os
from collections import OrderedDict
from copy import deepcopy

import cv2
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision as tv
from PIL import Image
from torch import linalg
from torch.nn import functional as F

In [2]:
def read_image(image_file):
    img = cv2.imread(image_file, cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if img is None:
        raise ValueError("Failed to read {}".format(image_file))
    return img

class IRPeople(data.Dataset):
    def __init__(self, root, annotation_file, transforms):
        self.root = root
        print(annotation_file)
        self.imlist = pd.read_csv(annotation_file)
        self.transforms = transforms

    def __getitem__(self, index):
        cv2.setNumThreads(6)

        impath, target= self.imlist.iloc[index]
        impath = impath.split('/')[-1].strip()

        full_imname = os.path.join(self.root, impath)
        img = read_image(full_imname)

        img = Image.fromarray(img)
        img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imlist)
    
def get_train_aug():
    train_augs = tv.transforms.Compose(
        [
            tv.transforms.Resize((224, 224)),
            tv.transforms.RandomVerticalFlip(),
            tv.transforms.RandomHorizontalFlip(),
            tv.transforms.ToTensor(),
            tv.transforms.Normalize(
                mean=[0.48145466, 0.4578275, 0.40821073],
                std=[0.26862954, 0.26130258, 0.27577711],
            ),
        ]
    )
    return train_augs

def get_dataloaders():
    print("Preparing train reader...")
    train_dataset = IRPeople(
        root=os.path.join('./images', "train"),
        annotation_file=os.path.join('./images', 'annotations.csv'),
        transforms=get_train_aug(),
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=32,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
        drop_last=True,
    )
    print("Done.")
    return train_loader

In [4]:
class ArcFace(nn.Module):
def __init__(self, cin, cout, s=30, m=0.5, stride=0.1, max_m=0.8):
        super().__init__()
        self.m = m
        self.s = s
        self.sin_m = torch.sin(torch.tensor(self.m))
        self.cos_m = torch.cos(torch.tensor(self.m))
        self.cout = cout
        self.fc = nn.Linear(cin, cout, bias=False)
        self.last_epoch = 0
        self.max_m = max_m
        self.m_s = stride

    def update(self, c_epoch):
        self.m = min(self.m + self.m_s * (c_epoch - self.last_epoch), self.max_m)
        self.last_epoch = c_epoch
        self.sin_m = torch.sin(torch.tensor(self.m))
        self.cos_m = torch.cos(torch.tensor(self.m))

    def forward(self, x, label=None):
        w_L2 = linalg.norm(self.fc.weight.detach(), dim=1, keepdim=True).T
        x_L2 = linalg.norm(x, dim=1, keepdim=True)
        cos = self.fc(x) / (x_L2 * w_L2)
        if label is not None:
            sin_m, cos_m = self.sin_m, self.cos_m
            one_hot = F.one_hot(label, num_classes=self.cout)
            sin = (1 - cos**2) ** 0.5
            angle_sum = cos * cos_m - sin * sin_m
            cos = angle_sum * one_hot + cos * (1 - one_hot)
            cos = cos * self.s
        return cos

class Classifier_model(nn.Module):
    def __init__(self):
        super(Classifier_model, self).__init__()
        self.model = tv.models.resnet50(pretrained=True)
        state_dict_model = torch.load("../model/ft_ResNet50/net_last.pth")
        new_state_dict = OrderedDict()
        for k, v in state_dict_model.items():
            if "model." in k:
                name = k.replace("model.", "", 1)
                new_state_dict[name] = v
        self.model.load_state_dict(new_state_dict)
        self.fc = ArcFace(
            1000,
            9,
            s=30,
            m=0.3,
            stride=0.05,
            max_m=0.8,
        )
    def forward(self, x, labels=None):
        x = self.model(x)
        x = self.fc(x, labels)
        return x


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 14)

In [ ]:
class IRPeopleModule(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = Classifier_model()
        self.loss_module = nn.CrossEntropyLoss()

    def forward(self, img, labels):
        return self.model(img, labels)

    def configure_optimizers(self):
        params = [
            {"params": self.model.model.parameters(), "lr": 1e-5},
            {"params": self.model.fc.parameters(), "lr": 1e-3},
        ]
        self.optimizer = torch.optim.AdamW(params, weight_decay=1e-2)
        return [self.optimizer]

    def training_step(self, batch, batch_idx):
        img, labels = batch
        preds = self.model(img, labels)
        loss = self.loss_module(preds, labels)
        acc = (preds.argmax(dim=-1) == labels).float().mean()
        # Logs the accuracy per epoch to tensorboard (weighted average over batches)
        self.log("train_acc", acc, on_step=True, on_epoch=True, prog_bar=True)
        self.model.fc.update(self.current_epoch)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=False)
        return loss  # Return tensor to call ".backward" on


In [ ]:
train_loader= get_dataloaders()

trainer = pl.Trainer(
    max_epochs=10,
    accelerator="gpu",
    precision=16,
    devices=[0],
)
model = IRPeopleModule()
trainer.fit(model, train_dataloaders=train_loader)

/home/luffy/miniconda3/envs/tracker/lib/python3.10/site-packages/lightning_fabric/connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Preparing train reader...
./images/annotations.csv
Done.


/home/luffy/miniconda3/envs/tracker/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/luffy/miniconda3/envs/tracker/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.htm

Epoch 1:  16%|█▌        | 96/614 [00:04<00:25, 20.53it/s, v_num=0, train_acc_step=0.938, train_acc_epoch=0.843] 

/home/luffy/miniconda3/envs/tracker/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
